In [ ]:
!unzip '/content/drive/MyDrive/Drowsiness project.zip' -d '/content/drive/MyDrive/Drowsiness_CNN'

Streaming output truncated to the last 5000 lines.
 extracting: /content/drive/MyDrive/Drowsiness_CNN/Drowsiness project/prepared_data/train/open_eyes/s0036_04905_1_0_1_0_0_01.png  
 extracting: /content/drive/MyDrive/Drowsiness_CNN/Drowsiness project/prepared_data/train/open_eyes/s0036_04906_1_0_1_0_0_01.png  
 extracting: /content/drive/MyDrive/Drowsiness_CNN/Drowsiness project/prepared_data/train/open_eyes/s0036_04907_1_0_1_1_0_01.png  
 extracting: /content/drive/MyDrive/Drowsiness_CNN/Drowsiness project/prepared_data/train/open_eyes/s0036_04908_1_0_1_0_0_01.png  
 extracting: /content/drive/MyDrive/Drowsiness_CNN/Drowsiness project/prepared_data/train/open_eyes/s0036_04909_1_0_1_1_0_01.png  
 extracting: /content/drive/MyDrive/Drowsiness_CNN/Drowsiness project/prepared_data/train/open_eyes/s0036_04910_1_0_1_0_0_01.png  
 extracting: /content/drive/MyDrive/Drowsiness_CNN/Drowsiness project/prepared_data/train/open_eyes/s0036_04911_1_0_1_0_0_01.png  
 extracting: /content/drive/MyDr

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [ ]:
tf.test.is_gpu_available()


True

In [ ]:
batchsize=8

In [ ]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'/content/drive/MyDrive/Drowsiness_CNN/Drowsiness project/prepared_data/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'/content/drive/MyDrive/Drowsiness_CNN/Drowsiness project/prepared_data/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 61113 images belonging to 2 classes.
Found 15277 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'/content/drive/MyDrive/Drowsiness_CNN/Drowsiness project/prepared_data/test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 8508 images belonging to 2 classes.


In [ ]:

bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[0][0]']

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau


In [ ]:
checkpoint = ModelCheckpoint(r'/content/drive/MyDrive/Drowsiness_CNN/Drowsiness project/models/model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, steps_per_epoch=train_data.samples // batchsize,
          validation_data=validation_data,
          validation_steps=validation_data.samples // batchsize,
          callbacks=callbacks,
          epochs=5)


Epoch 1/5
7638/7639 [============================>.] - ETA: 0s - loss: 0.2071 - accuracy: 0.9180
Epoch 1: val_loss improved from inf to 0.23787, saving model to /content/drive/MyDrive/Drowsiness_CNN/Drowsiness project/models/model.h5
7639/7639 [==============================] - 384s 50ms/step - loss: 0.2072 - accuracy: 0.9179 - val_loss: 0.2379 - val_accuracy: 0.9024 - lr: 0.0010
Epoch 2/5
7639/7639 [==============================] - ETA: 0s - loss: 0.1790 - accuracy: 0.9311
Epoch 2: val_loss did not improve from 0.23787
7639/7639 [==============================] - 359s 47ms/step - loss: 0.1790 - accuracy: 0.9311 - val_loss: 0.2611 - val_accuracy: 0.8989 - lr: 0.0010
Epoch 3/5
7639/7639 [==============================] - ETA: 0s - loss: 0.1681 - accuracy: 0.9349
Epoch 3: val_loss did not improve from 0.23787
7639/7639 [==============================] - 359s 47ms/step - loss: 0.1681 - accuracy: 0.9349 - val_loss: 0.2456 - val_accuracy: 0.8998 - lr: 0.0010
Epoch 4/5
7638/7639 [==========

In [ ]:
acc_tr, loss_tr = model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

<ipython-input-17-87a1e491e135>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.13315428793430328
0.9486525058746338


In [ ]:
acc_vr, loss_vr = model.evaluate(validation_data)
print(acc_vr)
print(loss_vr)

1910/1910 [==============================] - 71s 37ms/step - loss: 0.2466 - accuracy: 0.9046
0.24664436280727386
0.9045624136924744


In [ ]:
acc_test, loss_test = model.evaluate(test_data)
print(acc_tr)
print(loss_tr)

1064/1064 [==============================] - 23s 22ms/step - loss: 0.1336 - accuracy: 0.9492
0.13315428793430328
0.9486525058746338
